In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

Target:

    Add Data augmentation to improve accuracy

Result:

    1st run:
      Change : Added GAP layer with kernel size = 6
      Parameters : 8,838
      Train accuracy = 98.71%
      Test accuracy = 99.21%%

    2nd run : 
      Change : Batch normalization to every layer
      Parameters : 8,954
      Test accuracy : very low - 30%
 

Analysis:

    1st iteration - No overfitting, we can improve the model
    2nd iteration - Adding batch norm to all layers and removing dropout from 6th conv layer has reduced the accuracy drastically.
    


In [0]:
# Final network from Session 5 assignment
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.drop1 = nn.Dropout(0.1)                        #                       GRF - 1   J - 1
        self.conv1 = nn.Conv2d(1, 16, 3, padding=0)         #input - 28 OUtput - 26  J - 1  GRF - 3
        self.conv2 = nn.Conv2d(16, 10, 3, padding=0)        #input - 26 OUtput - 24  J - 1  GRF - 5 
        self.pool1 = nn.MaxPool2d(2, 2)                     #input - 24 OUtput - 12  J - 2  GRF - 7
        self.bnm2d1 = nn.BatchNorm2d(10) 
        self.conv3 = nn.Conv2d(10, 10, 3, padding=0)        #input - 12 OUtput - 10  J - 2  GRF - 11
        self.conv4 = nn.Conv2d(10, 16, 3, padding=0)        #input - 10 OUtput - 8   J - 2  GRF - 15
        # self.pool2 = nn.MaxPool2d(2, 2)
        self.bnm2d2 = nn.BatchNorm2d(16) 
        self.conv5 = nn.Conv2d(16, 16, 3, padding=0)        #input - 8  OUtput - 6   J - 2  GRF - 19
        self.conv6 = nn.Conv2d(16, 16, 3, padding=1)        #input - 6  OUtput - 6   J - 2  GRF - 23
        # self.conv7 = nn.Conv2d(16, 16, 3, padding=1)      #input - 6  OUtput - 6 
        self.gap = nn.AvgPool2d(kernel_size=6)
        self.conv8 = nn.Conv2d(16,10,1)                                             #J = 2  GRF - 23 

    def forward(self, x):
        x = self.bnm2d1(self.pool1(self.drop1(F.relu(self.conv2(self.drop1(F.relu(self.conv1(x))))))))
        x = self.bnm2d2(self.drop1(F.relu(self.conv4(self.drop1(F.relu(self.conv3(x)))))))
        x = self.conv6(self.drop1(F.relu(self.conv5(x))))
        # x = self.conv7(x)
        x = self.gap(x)
        x = self.conv8(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

# New Section

In [0]:
# !pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
           Dropout-2           [-1, 16, 26, 26]               0
            Conv2d-3           [-1, 10, 24, 24]           1,450
           Dropout-4           [-1, 10, 24, 24]               0
         MaxPool2d-5           [-1, 10, 12, 12]               0
       BatchNorm2d-6           [-1, 10, 12, 12]              20
            Conv2d-7           [-1, 10, 10, 10]             910
           Dropout-8           [-1, 10, 10, 10]               0
            Conv2d-9             [-1, 16, 8, 8]           1,456
          Dropout-10             [-1, 16, 8, 8]               0
      BatchNorm2d-11             [-1, 16, 8, 8]              32
           Conv2d-12             [-1, 16, 6, 6]           2,320
          Dropout-13             [-1, 16, 6, 6]               0
           Conv2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomRotation((-0.7,0.7), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
train_acc = []
test_acc = []
train_losses = []
test_losses = []

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0

    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        y_pred = model(data)

        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

        pred = y_pred.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)

        # pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
        train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
# Session 5 final model
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 15):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=0.3231932818889618 batch_id=468 Accuracy=72.75: 100%|██████████| 469/469 [00:15<00:00, 30.51it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.1551, Accuracy: 9530/10000 (95.30%)



loss=0.08738628774881363 batch_id=468 Accuracy=96.30: 100%|██████████| 469/469 [00:15<00:00, 30.44it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0833, Accuracy: 9754/10000 (97.54%)



loss=0.06012570858001709 batch_id=468 Accuracy=97.39: 100%|██████████| 469/469 [00:15<00:00, 30.73it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0304, Accuracy: 9908/10000 (99.08%)



loss=0.06741010397672653 batch_id=468 Accuracy=98.80: 100%|██████████| 469/469 [00:15<00:00, 30.26it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0297, Accuracy: 9910/10000 (99.10%)



loss=0.07848922163248062 batch_id=468 Accuracy=98.82: 100%|██████████| 469/469 [00:15<00:00, 30.35it/s]



Test set: Average loss: 0.0312, Accuracy: 9910/10000 (99.10%)



In [0]:
# def __init__(self):
#     super(Net, self).__init__()
#     self.drop1 = nn.Dropout(0.1)                        #                       GRF - 1   J - 1
#     self.conv1 = nn.Conv2d(1, 16, 3, padding=0)         #input - 28 OUtput - 26  J - 1  GRF - 3
#     self.conv2 = nn.Conv2d(16, 10, 3, padding=0)        #input - 26 OUtput - 24  J - 1  GRF - 5 
#     self.pool1 = nn.MaxPool2d(2, 2)                     #input - 24 OUtput - 12  J - 2  GRF - 7
#     self.bnm2d1 = nn.BatchNorm2d(10) 
#     self.conv3 = nn.Conv2d(10, 10, 3, padding=0)        #input - 12 OUtput - 10  J - 2  GRF - 11
#     self.conv4 = nn.Conv2d(10, 16, 3, padding=0)        #input - 10 OUtput - 8   J - 2  GRF - 15
#     # self.pool2 = nn.MaxPool2d(2, 2)
#     self.bnm2d2 = nn.BatchNorm2d(16) 
#     self.conv5 = nn.Conv2d(16, 16, 3, padding=0)        #input - 8  OUtput - 6   J - 2  GRF - 19
#     self.conv6 = nn.Conv2d(16, 16, 3, padding=1)        #input - 6  OUtput - 6   J - 2  GRF - 23
#     # self.conv7 = nn.Conv2d(16, 16, 3, padding=1)      #input - 6  OUtput - 6 
#     self.gap = nn.AvgPool2d(kernel_size=6)
#     self.conv8 = nn.Conv2d(16,10,1)                                             #J = 2  GRF - 23 

# def forward(self, x):
#     x = self.bnm2d1(self.pool1(self.drop1(F.relu(self.conv2(self.drop1(F.relu(self.conv1(x))))))))
#     x = self.bnm2d2(self.drop1(F.relu(self.conv4(self.drop1(F.relu(self.conv3(x)))))))
#     x = self.conv6(self.drop1(F.relu(self.conv5(x))))
#     # x = self.conv7(x)
#     x = self.gap(x)
#     x = self.conv8(x)
#     x = x.view(-1, 10)
#     return F.log_softmax(x)


model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 25):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=0.3072833716869354 batch_id=468 Accuracy=72.75: 100%|██████████| 469/469 [00:16<00:00, 28.90it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.1799, Accuracy: 9437/10000 (94.37%)



loss=0.07545644044876099 batch_id=468 Accuracy=96.31: 100%|██████████| 469/469 [00:16<00:00, 28.73it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0744, Accuracy: 9774/10000 (97.74%)



loss=0.06931450217962265 batch_id=468 Accuracy=97.42: 100%|██████████| 469/469 [00:16<00:00, 28.25it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0283, Accuracy: 9914/10000 (99.14%)



loss=0.02714567817747593 batch_id=468 Accuracy=99.11: 100%|██████████| 469/469 [00:16<00:00, 28.72it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0273, Accuracy: 9921/10000 (99.21%)



loss=0.027553394436836243 batch_id=468 Accuracy=99.06: 100%|██████████| 469/469 [00:16<00:00, 28.93it/s]



Test set: Average loss: 0.0291, Accuracy: 9922/10000 (99.22%)

